In [14]:
import pandas as pd
import numpy as np
import requests

# Your Alpha Vantage API Key
API_KEY = "6GMUD3MXU7HKTDSY"

# Fetch SPY daily data from Alpha Vantage
def fetch_spy_alpha_vantage(api_key):
    url = f"https://www.alphavantage.co/query"
    params = {
        "function": "TIME_SERIES_DAILY_ADJUSTED",
        "symbol": "SPY",
        "outputsize": "full",
        "apikey": api_key
    }
    r = requests.get(url, params=params)
    data = r.json()
    if "Time Series (Daily)" not in data:
        raise ValueError("❌ Failed to fetch SPY data from Alpha Vantage.")

    df = pd.DataFrame.from_dict(data["Time Series (Daily)"], orient="index")
    df = df.rename(columns={
        '1. open': 'open',
        '2. high': 'high',
        '3. low': 'low',
        '4. close': 'close',
        '5. adjusted close': 'adjusted_close',
        '6. volume': 'volume'
    })
    df.index = pd.to_datetime(df.index)
    df = df.sort_index()
    df = df[['adjusted_close']]
    return df

# Fetch SPY data
spy = fetch_spy_alpha_vantage(API_KEY)

# Print available data range
print(f"First available date: {spy.index.min().strftime('%Y-%m-%d')}")
print(f"Last available date: {spy.index.max().strftime('%Y-%m-%d')}")

# Filter for target range if possible
spy = spy.loc["1993-01-29":"2024-12-30"]

# Step 2: Calculate daily returns
spy['Prev_Close'] = spy['adjusted_close'].shift(1)
spy['Daily_Return'] = (spy['adjusted_close'] - spy['Prev_Close']) / spy['Prev_Close']

# Step 3: Strategy: Only hold if previous day was positive
spy['Prev_Return'] = spy['Daily_Return'].shift(1)
spy['Strategy_Return'] = np.where(spy['Prev_Return'] > 0, spy['Daily_Return'], 0)

# Step 4: Calculate cumulative returns
spy['Cumulative_Strategy_Return'] = (1 + spy['Strategy_Return']).cumprod()

# Step 5: Calculate total and annualized return
total_days = (spy.index[-1] - spy.index[0]).days
total_years = total_days / 365.25

final_cumulative_return = spy['Cumulative_Strategy_Return'].iloc[-1]
average_annual_return = (final_cumulative_return) ** (1/total_years) - 1

# Output results
print(f"✅ Final cumulative return (strategy): {final_cumulative_return:.2f}x")
print(f"✅ Average annual return: {average_annual_return * 100:.2f}%")


ValueError: ❌ Failed to fetch SPY data from Alpha Vantage.

In [15]:
import os
import pandas as pd
import investpy

# Define FOREX pairs and their Investing.com names
forex_pairs = {
    'EUR_USD': 'eur/usd',
    'USD_JPY': 'usd/jpy',
    'GBP_USD': 'gbp/usd',
    'AUD_USD': 'aud/usd',
    'USD_CAD': 'usd/cad'
}

# Define the date range for March 2025
start_date = '01/04/2025'  # DD/MM/YYYY
end_date = '30/04/2025'

# Folder to save data
os.makedirs('data', exist_ok=True)

# Initialize main DataFrame
combined_df = pd.DataFrame()

# Download and format data
for pair_name, pair in forex_pairs.items():
    print(f"Downloading {pair_name}...")
    try:
        data = investpy.get_currency_cross_historical_data(
            currency_cross=pair.upper(),
            from_date=start_date,
            to_date=end_date
        )
        data = data[['Open', 'High', 'Low', 'Close']]
        data.columns = [
            f'open_{pair_name}',
            f'high_{pair_name}',
            f'low_{pair_name}',
            f'close_{pair_name}'
        ]
        
        if combined_df.empty:
            combined_df = data.copy()
        else:
            combined_df = combined_df.join(data, how='outer')
    except Exception as e:
        print(f"Failed to download {pair_name}: {e}")

# Final formatting
combined_df.index.name = 'timestamp'
combined_df = combined_df.reset_index()

# Save to CSV
output_path = os.path.join('data', 'forex.csv')
combined_df.to_csv(output_path, index=False)

print(f"Data saved to {output_path}")

Data saved to data/forex.csv


VIX - investing.com

SPGSCI - investing.com

BTC - investing.com

Copper - https://www.macrotrends.net/1476/copper-prices-historical-chart-data

Oil - https://www.macrotrends.net/2516/wti-crude-oil-prices-10-year-daily-chart

MOEX - https://www.investing.com/indices/mcx-historical-data?utm_source=chatgpt.com

STOXX 600 - https://www.investing.com/indices/stoxx-600?utm_source=chatgpt.com

SSE - https://www.investing.com/indices/shanghai-composite-historical-data


In [35]:
import pandas as pd
from fredapi import Fred
import os
from datetime import datetime

def get_treasury_10y_rates(start_date, end_date):
    """
    Fetch US Treasury 10-year daily rates from FRED API and save to CSV
    """
    # Initialize FRED API
    fred = Fred(api_key='65d4ac185aed89dd59c0f551a6db85b0')
    
    # Series ID for 10-Year Treasury Constant Maturity Rate
    series_id = 'DGS10'  # FRED code for 10-Year Treasury Rate
    
    try:
        # Create data directory if it doesn't exist
        os.makedirs('data', exist_ok=True)
        
        # Fetch the data
        data = fred.get_series(series_id, start_date, end_date)
        
        if data.empty:
            current_year = datetime.now().year
            print(f"No data available for {start_date} to {end_date}")
            if int(start_date[:4]) > current_year:
                print(f"Note: Future data (April {start_date[:4]}) isn't available yet")
            return False
            
        # Convert to DataFrame
        df = pd.DataFrame(data, columns=['Rate'])
        df.index.name = 'Date'
        df['Rate'] = df['Rate'].astype(float)
        
        # Save to CSV
        csv_path = 'data/treasury_10y.csv'
        df.to_csv(csv_path)
        print(f"Successfully saved 10-year Treasury rates to {csv_path}")
        print(f"Data range: {df.index[0].date()} to {df.index[-1].date()}")
        return True
        
    except Exception as e:
        print(f"An error occurred: {e}")
        return False

# Define date range for April 2025
start_date = '2025-04-01'
end_date = '2025-04-30'

# Get and save the data
success = get_treasury_10y_rates(start_date, end_date)

if not success:
    # Example fallback to available data
    print("\nTrying available historical data for demonstration...")
    get_treasury_10y_rates('2024-04-01', '2024-04-30')

Successfully saved 10-year Treasury rates to data/treasury_10y.csv
Data range: 2025-04-01 to 2025-04-30


In [52]:
import os
import requests
import pandas as pd
from datetime import datetime
from pandas.tseries.offsets import BDay

# Configuration
START_DATE = "2025-04-01"
END_DATE = (datetime.today() - BDay(1)).strftime('%Y-%m-%d')
OUTPUT_FILE = "data/bitcoin.csv"
COINGECKO_URL = (
    "https://api.coingecko.com/api/v3/coins/bitcoin/market_chart"
    "?vs_currency=usd&days=max&interval=daily"
)

# Ensure output directory exists
os.makedirs(os.path.dirname(OUTPUT_FILE), exist_ok=True)

# Request data
print("📡 Fetching Bitcoin historical data...")
response = requests.get(COINGECKO_URL)
data = response.json()

# Check for errors in response
if 'prices' not in data or 'total_volumes' not in data:
    raise Exception(f"❌ API error or rate limit. Response was: {data}")

# Convert price data
prices = pd.DataFrame(data['prices'], columns=['timestamp', 'Close'])
prices['timestamp'] = pd.to_datetime(prices['timestamp'], unit='ms').dt.floor('D')

# Use Close as proxy for OHLC
prices['Open'] = prices['Close']
prices['High'] = prices['Close']
prices['Low'] = prices['Close']

# Convert volume data
volumes = pd.DataFrame(data['total_volumes'], columns=['timestamp', 'Volume'])
volumes['timestamp'] = pd.to_datetime(volumes['timestamp'], unit='ms').dt.floor('D')

# Merge
df = prices.merge(volumes, on='timestamp', how='left')

# Filter date range
df = df[(df['timestamp'] >= START_DATE) & (df['timestamp'] <= END_DATE)]

# Format and save
df = df[['timestamp', 'Open', 'High', 'Low', 'Close', 'Volume']]
df[['Open', 'High', 'Low', 'Close']] = df[['Open', 'High', 'Low', 'Close']].round(2)
df['Volume'] = df['Volume'].round(2)
df['timestamp'] = df['timestamp'].dt.strftime('%Y-%m-%d')
df.to_csv(OUTPUT_FILE, index=False)

print(f"✅ Bitcoin data saved to {OUTPUT_FILE}")


📡 Fetching Bitcoin historical data...


Exception: ❌ API error or rate limit. Response was: {'error': {'status': {'timestamp': '2025-05-04T17:27:37.840+00:00', 'error_code': 10012, 'error_message': 'Your request exceeds the allowed time range. Public API users are limited to querying historical data within the past 365 days. Upgrade to a paid plan to enjoy full historical data access: https://www.coingecko.com/en/api/pricing. '}}}

In [50]:
import os
import pandas as pd
import requests
from bs4 import BeautifulSoup
from datetime import datetime

# Configuration
URL = "https://finance.yahoo.com/quote/%5EGSPC/history"
START_DATE = "2025-04-01"
OUTPUT_FILE = "data/sp500.csv"
HEADERS = {
    "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36"
}

# Ensure output directory exists
os.makedirs(os.path.dirname(OUTPUT_FILE), exist_ok=True)

# Request page content with headers
response = requests.get(URL, headers=HEADERS)
if response.status_code == 429:
    raise Exception("❌ HTTP 429: Too Many Requests. You're being rate-limited by Yahoo.")

soup = BeautifulSoup(response.text, "lxml")
tables = soup.find_all("table")

if not tables:
    raise Exception("❌ No table found. Yahoo Finance layout may have changed.")

# Parse first table into DataFrame
df = pd.read_html(str(tables[0]))[0]

# Debug: Print column names to confirm structure
print("📋 Available columns:", list(df.columns))

# Try to find the closest match to the 'Close' column
close_column = next((col for col in df.columns if "Close" in col), None)
if close_column is None:
    raise Exception("❌ 'Close' column not found in table. Yahoo layout likely changed.")

# Drop rows without numeric Close prices (e.g., 'Dividend', 'Split', NaN)
df = df[df[close_column].apply(lambda x: isinstance(x, (int, float)) or str(x).replace('.', '', 1).isdigit())]

# Convert and clean dates
df['Date'] = pd.to_datetime(df['Date'], format='mixed', errors='coerce')
df = df[df['Date'] >= pd.to_datetime(START_DATE)]

# Rename and format
df = df.rename(columns={
    'Date': 'timestamp',
    'Open': 'Open',
    'High': 'High',
    'Low': 'Low',
    close_column: 'Close',
    'Volume': 'Volume'
})
df = df[['timestamp', 'Open', 'High', 'Low', 'Close', 'Volume']]
df[['Open', 'High', 'Low', 'Close']] = df[['Open', 'High', 'Low', 'Close']].astype(float).round(2)
df['Volume'] = df['Volume'].astype(str).str.replace(',', '').astype(float)
df['timestamp'] = df['timestamp'].dt.strftime('%Y-%m-%d')
df = df.sort_values('timestamp')

# Save to CSV
df.to_csv(OUTPUT_FILE, index=False)
print(f"✅ S&P 500 data scraped and saved to {OUTPUT_FILE}")


📋 Available columns: ['Date', 'Open', 'High', 'Low', 'Close Close price adjusted for splits.', 'Adj Close Adjusted close price adjusted for splits and dividend and/or capital gain distributions.', 'Volume']
✅ S&P 500 data scraped and saved to data/sp500.csv


/var/folders/sd/x6b24fj17sg_pd8tj67xfp4r0000gn/T/ipykernel_18014/3001098383.py:30: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(tables[0]))[0]


In [54]:
import os
import pandas as pd
import requests
from bs4 import BeautifulSoup
from datetime import datetime

# Configuration
URL = "https://finance.yahoo.com/quote/CL%3DF/history"
START_DATE = "2025-04-01"
END_DATE = "2025-04-30"
OUTPUT_FILE = "data/oil.csv"
HEADERS = {
    "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36"
}

# Ensure output directory exists
os.makedirs(os.path.dirname(OUTPUT_FILE), exist_ok=True)

# Request page content with headers
response = requests.get(URL, headers=HEADERS)
if response.status_code == 429:
    raise Exception("❌ HTTP 429: Too Many Requests. You're being rate-limited by Yahoo.")
if response.status_code != 200:
    raise Exception(f"❌ HTTP {response.status_code}: Failed to load data.")

# Parse table using BeautifulSoup
soup = BeautifulSoup(response.text, "lxml")
tables = soup.find_all("table")

if not tables:
    raise Exception("❌ No table found. Yahoo Finance layout may have changed.")

# Convert to DataFrame
df = pd.read_html(str(tables[0]))[0]

# Print column names for verification
print("📋 Available columns:", list(df.columns))

# Find the column that contains 'Close'
close_column = next((col for col in df.columns if "Close" in col), None)
if close_column is None:
    raise Exception("❌ 'Close' column not found. Layout likely changed.")

# Remove non-price rows (e.g., dividends, splits)
df = df[df[close_column].apply(lambda x: isinstance(x, (int, float)) or str(x).replace('.', '', 1).isdigit())]

# Parse and filter dates
df['Date'] = pd.to_datetime(df['Date'], format='mixed', errors='coerce')
df = df[(df['Date'] >= pd.to_datetime(START_DATE)) & (df['Date'] <= pd.to_datetime(END_DATE))]

# Rename and reformat
df = df.rename(columns={
    'Date': 'timestamp',
    'Open': 'Open',
    'High': 'High',
    'Low': 'Low',
    close_column: 'Close',
    'Volume': 'Volume'
})
df = df[['timestamp', 'Open', 'High', 'Low', 'Close', 'Volume']]
df[['Open', 'High', 'Low', 'Close']] = df[['Open', 'High', 'Low', 'Close']].astype(float).round(4)
df['Volume'] = df['Volume'].astype(str).str.replace(',', '').astype(float)
df['timestamp'] = df['timestamp'].dt.strftime('%Y-%m-%d')
df = df.sort_values('timestamp')

# Save to CSV
df.to_csv(OUTPUT_FILE, index=False)
print(f"✅ Oil price data saved to {OUTPUT_FILE}")


📋 Available columns: ['Date', 'Open', 'High', 'Low', 'Close Close price adjusted for splits.', 'Adj Close Adjusted close price adjusted for splits and dividend and/or capital gain distributions.', 'Volume']
✅ Oil price data saved to data/oil.csv


/var/folders/sd/x6b24fj17sg_pd8tj67xfp4r0000gn/T/ipykernel_18014/2462068041.py:34: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(tables[0]))[0]


In [56]:
import pandas as pd
from pytrends.request import TrendReq
import os

# Set up pytrends
pytrends = TrendReq(hl='en-US', tz=360)

# Define search terms
keywords = ['sp500', 'SPX', 'index fund', 'ETF']
start_date = '2025-04-01'
end_date = '2025-04-30'
geo = ''  # Can set to 'US' for US-specific trends

# Build payload
pytrends.build_payload(keywords, timeframe=f'{start_date} {end_date}', geo=geo)

# Fetch interest over time
df = pytrends.interest_over_time()

# Drop 'isPartial' column if it exists
if 'isPartial' in df.columns:
    df = df.drop(columns=['isPartial'])

# Compute 7-day rolling average
df_rolling = df.rolling(window=7, min_periods=1).mean()

# Reset index and rename for CSV output
df_rolling = df_rolling.reset_index()
df_rolling.rename(columns={'date': 'timestamp'}, inplace=True)

# Ensure output folder exists
os.makedirs('data', exist_ok=True)

# Save to CSV
df_rolling.to_csv('data/google_trends.csv', index=False)

print("✅ Google Trends data saved to data/google_trends.csv")


✅ Google Trends data saved to data/google_trends.csv
